In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

class UF:
  def __init__(self, n):
    self.p = list(range(n))

  def find(self, a):
    a_ = a
    while a != self.p[a]:
      a = self.p[a]
    while a_ != a:
      self.p[a_], a_ = a, self.p[a_]
    return a

  def merge(self, a, b):
    self.p[self.find(b)] = self.find(a)

def sol(N, M) :
  Q = [] #쿼리
  E = [] #간선
  for _ in range(M) :
    q, *l = input().split()
    l = tuple(map(int, l))
    Q.append((q, l))
    if q == b'!' :
      E.append(l)
  
  uf = UF(N+1)
  IN = [[] for _ in range(N+1)] 
  OUT = [[] for _ in range(N+1)] 
  for a, b, w in E :
    uf.merge(b, a)
    IN[b].append((a, w))
    OUT[a].append(b)
  
  C = {} #그룹 내에서 가장 무거운 샘플으로부터 상대적인 무게 차이의 합
  def dfs(r) :
    S = [(r, 0)]
    C[r] = 0 #가장 무거운 샘플은 무게가 0
    while S :
      v, w = S.pop()
      for u, w2 in IN[v] :
        if u in C: continue
        C[u] = w + w2
        S.append((u, C[u]))

  for u, out in enumerate(OUT[1:], 1) :
    if out: continue
    dfs(u)

  ans = []
  uf = UF(N+1) #'지금까지 측정한 것'을 시뮬레이션 해야하므로, UF를 초기화한다.
  for q, l in Q: 
    match q :
      case b'!' :
        a, b, w = l
        uf.merge(b, a)
      case _ :
        a, b = l
        if uf.find(a) != uf.find(b) :
          ans.append('UNKNOWN')
        else :
          ans.append(str(C[a] - C[b]))
  
  sys.stdout.write('\n'.join(ans) + "\n")

while (x := input().split()) != [b'0', b'0'] :
  sol(*map(int, x))

- 잘 만든거 같은데 WA가 난다. 아무래도 무슨 반례가 있을지도 모른다.
  - 무게차이는 0일 수 있다. 이 부분에서 뭔가 있는게아닐까?
  - 그런것 같긴 하다. a b 0, b a 0 이런식으로 들어오면 측정에 일관성이 있기도 하고, 정답도 나오지 않는다.

In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

class UF:
  def __init__(self, n):
    self.p = list(range(n))

  def find(self, a):
    a_ = a
    while a != self.p[a]:
      a = self.p[a]
    while a_ != a:
      self.p[a_], a_ = a, self.p[a_]
    return a

  def merge(self, a, b):
    self.p[self.find(b)] = self.find(a)

def sol(N, M) :
  Q = [] #쿼리
  E = [] #간선
  for _ in range(M) :
    q, *l = input().split()
    l = tuple(map(int, l))
    Q.append((q, l))
    if q == b'!' :
      E.append(l)
  
  uf = UF(N+1)
  IN = [[] for _ in range(N+1)] 
  OUT = [[] for _ in range(N+1)] 
  for a, b, w in E :
    if uf.find(a) == uf.find(b) : continue #무게 차이가 0이라면 a, b와 b,a가 둘다 나올 수 있다.
    uf.merge(b, a)
    IN[b].append((a, w))
    OUT[a].append(b)
  
  C = {} #그룹 내에서 가장 무거운 샘플으로부터 상대적인 무게 차이의 합
  def dfs(r) :
    S = [(r, 0)]
    C[r] = 0 #가장 무거운 샘플은 무게가 0
    while S :
      v, w = S.pop()
      for u, w2 in IN[v] :
        if u in C: continue
        C[u] = w + w2
        S.append((u, C[u]))

  for u, out in enumerate(OUT[1:], 1) :
    if out: continue
    dfs(u)

  ans = []
  uf = UF(N+1) #'지금까지 측정한 것'을 시뮬레이션 해야하므로, UF를 초기화한다.
  for q, l in Q: 
    match q :
      case b'!' :
        a, b, w = l
        if uf.find(a) == uf.find(b) : continue
        uf.merge(b, a)
      case _ :
        a, b = l
        if uf.find(a) != uf.find(b) :
          ans.append('UNKNOWN')
        else :
          ans.append(str(C[a] - C[b]))
  
  sys.stdout.write('\n'.join(ans) + "\n")

while (x := input().split()) != [b'0', b'0'] :
  sol(*map(int, x))

- 여전히 WA가 뜬다. 사실 위의 반례라면 RE가 떠야하는데, 그렇지 않은 것 부터 예상하곤 있었다.
  - 반례) 1 3 1, 1 2 1 Union find를 하는데에 있어서 상당한 문제가 있었다. 부모가 바뀔 수 있다는 것.

In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readlinea

class UF:
  def __init__(self, n):
    self.P = list(range(n))
    self.s = [0] * n #자신의 최종 부모까지의 무게 차이

  def find(self, A):
    a = A
    while A != self.P[A]:
      A = self.P[A]
    
    tmp = 0 #현재 노드에서 갱신해줘야 하는 무게의 차이의 합
    _a = a
    while a != A:
      tmp += self.s[a] #최종 부모가 갱신되기 이전에, 그 부모가 최종 부모까지의 무게차이라고 기록했던 값을 누적
      a = self.P[a]
    
    a = _a
    while a != A :
      self.s[a] = tmp #각 부모마다 최종 부모까지의 무게 차이를 갱신
      tmp -= self.s[a]
      self.P[a], a = A, self.P[a]
    return A

  def union(self, a, b, w):
    a, b = self.find(a), self.find(b)
    if a != b:
      if self.s[a] < self.s[b]:
        a, b = b, a

      self.P[b] = a
      self.s[a] += self.s[b]

def sol(N, M) :
  Q = [] #쿼리
  E = [] #간선
  for _ in range(M) :
    q, *l = input().split()
    l = tuple(map(int, l))
    Q.append((q, l))
    if q == b'!' :
      E.append(l)
  
  uf = UF(N+1)
  IN = [[] for _ in range(N+1)] 
  OUT = [[] for _ in range(N+1)] 
  for a, b, w in E :
    if uf.find(a) == uf.find(b) : continue
    uf.merge(b, a)
    IN[b].append((a, w))
    OUT[a].append(b)
  
  C = {} #그룹 내에서 가장 무거운 샘플으로부터 상대적인 무게 차이의 합
  def dfs(r) :
    S = [(r, 0)]
    C[r] = 0 #가장 무거운 샘플은 무게가 0
    while S :
      v, w = S.pop()
      for u, w2 in IN[v] :
        if u in C: continue
        C[u] = w + w2
        S.append((u, C[u]))

  for u, out in enumerate(OUT[1:], 1) :
    if out: continue
    dfs(u)

  ans = []
  uf = UF(N+1) #'지금까지 측정한 것'을 시뮬레이션 해야하므로, UF를 초기화한다.
  for q, l in Q: 
    match q :
      case b'!' :
        a, b, w = l
        if uf.find(a) == uf.find(b) : continue
        uf.merge(b, a)
      case _ :
        a, b = l
        if uf.find(a) != uf.find(b) :
          ans.append('UNKNOWN')
        else :
          ans.append(str(C[a] - C[b]))
  
  sys.stdout.write('\n'.join(ans) + "\n")

while (x := input().split()) != [b'0', b'0'] :
  sol(*map(int, x))

- 살려줘..